<a href="https://colab.research.google.com/github/JessemanGray/GraphUp/blob/main/GraphUp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install dash

In [6]:
pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [8]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import open3d as o3d
import numpy as np

# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import open3d as o3d
import numpy as np

# Function to read .ply file and return vertices and triangles
def get_mesh_data(filepath):
    mesh = o3d.io.read_triangle_mesh(filepath)
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    return vertices, triangles

# Function to create Mesh3d object
def create_mesh3d(vertices, triangles, color, opacity, name):
    return go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=triangles[:, 0],
        j=triangles[:, 1],
        k=triangles[:, 2],
        color=color,
        opacity=opacity,
        name=name
    )

# Read .ply files for base model and overlays
base_vertices, base_triangles = get_mesh_data(model_path)
mech_vertices, mech_triangles = get_mesh_data(mech_path)
elec_vertices, elec_triangles = get_mesh_data(elec_path)
plumb_vertices, plumb_triangles = get_mesh_data(plumb_path)

# Define Dash app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='3d-plot'),
    html.Label('Mechanical Opacity Slider'),
    dcc.Slider(id='mech-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Electrical Opacity Slider'),
    dcc.Slider(id='elec-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Plumbing Opacity Slider'),
    dcc.Slider(id='plumb-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Toggle Visibility'),
    dcc.Checklist(
        id='visibility-checklist',
        options=[
            {'label': 'Mechanical', 'value': 'mech'},
            {'label': 'Electrical', 'value': 'elec'},
            {'label': 'Plumbing', 'value': 'plumb'}
        ],
        value=['mech', 'elec', 'plumb']
    ),
    html.Label('Select Model'),
    dcc.Dropdown(
        id='model-dropdown',
        options=[
            {'label': 'Base Model', 'value': 'base'},
            {'label': 'Mechanical', 'value': 'mech'},
            {'label': 'Electrical', 'value': 'elec'},
            {'label': 'Plumbing', 'value': 'plumb'}
        ],
        value='base'
    )
])

@app.callback(
    Output('3d-plot', 'figure'),
    [Input('mech-opacity-slider', 'value'),
     Input('elec-opacity-slider', 'value'),
     Input('plumb-opacity-slider', 'value'),
     Input('visibility-checklist', 'value'),
     Input('model-dropdown', 'value')]
)
def update_graph(mech_opacity, elec_opacity, plumb_opacity, visibility, selected_model):
    traces = []

    if 'base' in selected_model:
        traces.append(create_mesh3d(base_vertices, base_triangles, 'lightyellow', 0.5, 'Base Model'))

    if 'mech' in visibility and 'mech' in selected_model:
        traces.append(create_mesh3d(mech_vertices, mech_triangles, 'orange', mech_opacity, 'Mechanical'))

    if 'elec' in visibility and 'elec' in selected_model:
        traces.append(create_mesh3d(elec_vertices, elec_triangles, 'hotpink', elec_opacity, 'Electrical'))

    if 'plumb' in visibility and 'plumb' in selected_model:
        traces.append(create_mesh3d(plumb_vertices, plumb_triangles, 'turquoise', plumb_opacity, 'Plumbing'))

    fig = go.Figure(
        data=traces,
        layout=go.Layout(
            autosize=False,
            width=800,
            height=400,
            plot_bgcolor='black',
            paper_bgcolor='black',
            font=dict(size=16),
            scene=dict(
                xaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False),
                zaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False)
            )
        )
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

# Function to read .ply file and return vertices and triangles
def get_mesh_data(filepath):
    mesh = o3d.io.read_triangle_mesh(filepath)
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    return vertices, triangles

# Function to create Mesh3d object
def create_mesh3d(vertices, triangles, color, opacity, name):
    return go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=triangles[:, 0],
        j=triangles[:, 1],
        k=triangles[:, 2],
        color=color,
        opacity=opacity,
        name=name
    )

# Read .ply files for base model and overlays
base_vertices, base_triangles = get_mesh_data(model_path)
mech_vertices, mech_triangles = get_mesh_data(mech_path)
elec_vertices, elec_triangles = get_mesh_data(elec_path)
plumb_vertices, plumb_triangles = get_mesh_data(plumb_path)

# Define Dash app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='3d-plot'),
    html.Label('Mechanical Opacity Slider'),
    dcc.Slider(id='mech-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Electrical Opacity Slider'),
    dcc.Slider(id='elec-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Plumbing Opacity Slider'),
    dcc.Slider(id='plumb-opacity-slider', min=0, max=1, step=0.1, value=0.5),
    html.Label('Toggle Visibility'),
    dcc.Checklist(
        id='visibility-checklist',
        options=[
            {'label': 'Mechanical', 'value': 'mech'},
            {'label': 'Electrical', 'value': 'elec'},
            {'label': 'Plumbing', 'value': 'plumb'}
        ],
        value=['mech', 'elec', 'plumb']
    ),
    html.Label('Select Model'),
    dcc.Dropdown(
        id='model-dropdown',
        options=[
            {'label': 'Base Model', 'value': 'base'},
            {'label': 'Mechanical', 'value': 'mech'},
            {'label': 'Electrical', 'value': 'elec'},
            {'label': 'Plumbing', 'value': 'plumb'}
        ],
        value='base'
    )
])

@app.callback(
    Output('3d-plot', 'figure'),
    [Input('mech-opacity-slider', 'value'),
     Input('elec-opacity-slider', 'value'),
     Input('plumb-opacity-slider', 'value'),
     Input('visibility-checklist', 'value'),
     Input('model-dropdown', 'value')]
)
def update_graph(mech_opacity, elec_opacity, plumb_opacity, visibility, selected_model):
    traces = []

    if 'base' in selected_model:
        traces.append(create_mesh3d(base_vertices, base_triangles, 'lightyellow', 0.5, 'Base Model'))

    if 'mech' in visibility and 'mech' in selected_model:
        traces.append(create_mesh3d(mech_vertices, mech_triangles, 'orange', mech_opacity, 'Mechanical'))

    if 'elec' in visibility and 'elec' in selected_model:
        traces.append(create_mesh3d(elec_vertices, elec_triangles, 'hotpink', elec_opacity, 'Electrical'))

    if 'plumb' in visibility and 'plumb' in selected_model:
        traces.append(create_mesh3d(plumb_vertices, plumb_triangles, 'turquoise', plumb_opacity, 'Plumbing'))

    fig = go.Figure(
        data=traces,
        layout=go.Layout(
            autosize=False,
            width=800,
            height=400,
            plot_bgcolor='black',
            paper_bgcolor='black',
            font=dict(size=16),
            scene=dict(
                xaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False),
                zaxis=dict(showbackground=False, title='', showgrid=False, zeroline=False, showticklabels=False)
            )
        )
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

NameError: name 'model_path' is not defined